In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationBufferMemory(return_messages=True)

examples_answer = [
    {"question": "탑건", "answer": "🛩️👨‍✈️🔥"},
    {"question": "대부", "answer": "👨‍👨‍👦🔫🍝"},
]

movie_prompt = ChatPromptTemplate.from_messages([
    ("human", "{question}"),
    ("ai", "{answer}"),
])

example_movie_prompt = FewShotChatMessagePromptTemplate(
    examples=examples_answer,
    example_prompt=movie_prompt,
)

prompt = ChatPromptTemplate.from_messages([
    ("system", """
        When the human asks for the title of a movie, answer it with three emoticons. 
        If the human asks a general question, not the title of the movie, answer it in Korean.
        Below is a few-shot example of how to answer movie-related questions:
    """),
    example_movie_prompt,
    ("system", "Now, below is the actual conversation history."),
    MessagesPlaceholder(variable_name="history"), 
    ("human", "{question}")
])

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context({"input": question}, {"output": result.content})  
    print(result.content)  


invoke_chain("검은수녀들")  
invoke_chain("말할 수 없는 비밀") 
invoke_chain("이 질문은 영화제목은 아니고 내가 체인에 첫 번째로 물어본 영화제목이 뭐였어?")

👩‍🦰🔪👻
🤫💔🎹
미안해요, 그 질문에 대한 답변을 제공할 수 없어요.


In [48]:
invoke_chain("왜 답변할 수 없어?")

죄송하지만, 그 질문에 대한 답변을 제공할 수 없습니다.


마지막 대답이 왜 계속 탑건이 나오는지 알 수 없음!